In [1]:
import cx_Oracle
import json
import pandas as pd

user_data = json.load(open('keys.json'))
connection = cx_Oracle.connect(user=user_data['username'], password=user_data['password'],
                                dsn="tarea1_high")

In [4]:
sql = "SELECT * FROM ADMIN.BOLETA"
df_ora = pd.read_sql(sql, con=connection)
df_ora.head()

,id,ID_CLIENTE,ID_DIRECCION,ESTADO,FECHA_COMPRA,MEDIO_PAGO,FECHA_AGENDADA
0,7269,20,215,Cumplido,2019-08-28,Deposito Banco de Credito e Inversiones,None
1,7287,20,215,Cumplido,2020-06-01,Deposito Banco de Credito e Inversiones,None
2,7740,19,215,Cumplido,2017-09-02,Deposito Banco de Credito e Inversiones,None
3,8017,20,215,Cumplido,2019-10-12,Deposito Banco de Credito e Inversiones,None
4,8047,20,215,Cumplido,2019-12-10,Deposito Banco de Credito e Inversiones,None
